In [3]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Create a date range for 12 months (365 days)
date_range = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# Create a dictionary with sample data
data = {
    'Date': date_range, # Use the previously created date range
    'Product': np.random.choice(['A','B','C','D'], size=len(date_range)), # Randomly choose products A, B, C or D
    'Sales': np.random.randint(100, 1000, size = len(date_range)), # Generate random sales between 100 and 999
    'Revenue': np.random.uniform(1000, 5000, size=len(date_range)) # Generate random revenue between 1000 and 5000
}

# Create a pandas DataFrame with the generated data
df = pd.DataFrame(data)

# Display the DataFrame
df

,Date,Product,Sales,Revenue
0,2023-01-01,C,514,3247.466769
1,2023-01-02,D,397,1954.387439
2,2023-01-03,A,710,3719.379120
3,2023-01-04,C,362,3959.635042
4,2023-01-05,C,863,1952.944610
...,...,...,...,...
360,2023-12-27,D,647,4514.516461
361,2023-12-28,A,195,2981.694506
362,2023-12-29,A,763,3965.843645
363,2023-12-30,B,762,3292.603397
